# Osu Country Codes

Loads the list of countries linking their name to their country code.

In [1]:
from bs4 import BeautifulSoup
import json
import requests

## Osu page

In [2]:
url = "https://osu.ppy.sh/rankings/osu/performance"


## Use playwright on the project

In [3]:
from playwright.async_api import async_playwright

savePath = "out/osu_ranking.html"
async def main():
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = False)
    page = await browser.new_page()
    await page.goto(url)
    await page.wait_for_timeout(1000)
    htmlContent = await page.content()
    with open(savePath, 'w', encoding='utf-8') as file:
        file.write(htmlContent)

    await browser.close()

await main()

In [4]:

with open(savePath, 'r') as file:
    html_contents = file.read()

In [5]:
parsed = BeautifulSoup(html_contents, 'html.parser')

In [6]:
from urllib.parse import urlparse, parse_qs

def getCountryParam(url):
    parsed_url = urlparse(url)
    # Get the query parameters as a dictionary
    query_parameters = parse_qs(parsed_url.query)
    # Get the value of the 'country' parameter
    country = query_parameters.get("country", [""])[0]
    return country

content = parsed.find(class_ = "select-options")

countries = content.find_all("a")

nameToCodeOsu = {}

for countryDropdown in countries:
    countryName = countryDropdown.text
    link = countryDropdown.get("href")
    countryCode = getCountryParam(link)
    if countryCode != "":
        nameToCodeOsu[countryName] = countryCode
    
print(nameToCodeOsu)

{'Afghanistan': 'AF', 'Aland Islands': 'AX', 'Albania': 'AL', 'Algeria': 'DZ', 'American Samoa': 'AS', 'Andorra': 'AD', 'Angola': 'AO', 'Anguilla': 'AI', 'Antigua and Barbuda': 'AG', 'Argentina': 'AR', 'Armenia': 'AM', 'Aruba': 'AW', 'Asia/Pacific Region': 'AP', 'Australia': 'AU', 'Austria': 'AT', 'Azerbaijan': 'AZ', 'Bahamas': 'BS', 'Bahrain': 'BH', 'Bangladesh': 'BD', 'Barbados': 'BB', 'Belarus': 'BY', 'Belgium': 'BE', 'Belize': 'BZ', 'Benin': 'BJ', 'Bermuda': 'BM', 'Bhutan': 'BT', 'Bolivia': 'BO', 'Bosnia and Herzegovina': 'BA', 'Botswana': 'BW', 'Bouvet Island': 'BV', 'Brazil': 'BR', 'British Indian Ocean Territory': 'IO', 'Brunei': 'BN', 'Bulgaria': 'BG', 'Burkina Faso': 'BF', 'Burundi': 'BI', 'Cabo Verde': 'CV', 'Cambodia': 'KH', 'Cameroon': 'CM', 'Canada': 'CA', 'Caribbean Netherlands': 'BQ', 'Cayman Islands': 'KY', 'Central African Republic': 'CF', 'Chad': 'TD', 'Chile': 'CL', 'China': 'CN', 'Christmas Island': 'CX', 'Cocos (Keeling) Islands': 'CC', 'Colombia': 'CO', 'Comoros':

In [7]:
json_data = json.dumps(
    nameToCodeOsu, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/osu_countries.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)

JSON data has been saved to out/osu_countries.json


## Filter to countries available in osu world

In [8]:
file_path = "out/osu_countries.json"

with open(file_path, "r", encoding="utf-8") as json_file:
    nameToCodeOsu = json.load(json_file)

In [9]:
def fetchData(api_url):
    try:
        response = requests.get(api_url)

        if response.status_code == 200:

            data = response.json()
            return data
        else:
            print(f"Request failed with status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)


In [10]:
countries: dict = fetchData("https://osuworld.octo.moe/locales/en/countries.json")

In [11]:
countriesCodes = countries.keys()


In [12]:
nameToCodeOsuFiltered = {k: v for k, v in nameToCodeOsu.items() if v in countriesCodes}
print(nameToCodeOsuFiltered)

{'Argentina': 'AR', 'Australia': 'AU', 'Austria': 'AT', 'Belarus': 'BY', 'Belgium': 'BE', 'Brazil': 'BR', 'Bulgaria': 'BG', 'Canada': 'CA', 'Chile': 'CL', 'China': 'CN', 'Colombia': 'CO', 'Czechia': 'CZ', 'Denmark': 'DK', 'Estonia': 'EE', 'Finland': 'FI', 'France': 'FR', 'Germany': 'DE', 'Greece': 'GR', 'Hungary': 'HU', 'Indonesia': 'ID', 'Ireland': 'IE', 'Israel': 'IL', 'Italy': 'IT', 'Japan': 'JP', 'Kazakhstan': 'KZ', 'Latvia': 'LV', 'Lithuania': 'LT', 'Malaysia': 'MY', 'Mexico': 'MX', 'Netherlands': 'NL', 'New Zealand': 'NZ', 'Norway': 'NO', 'Peru': 'PE', 'Philippines': 'PH', 'Poland': 'PL', 'Portugal': 'PT', 'Romania': 'RO', 'Russian Federation': 'RU', 'Singapore': 'SG', 'Slovakia': 'SK', 'South Korea': 'KR', 'Spain': 'ES', 'Sweden': 'SE', 'Switzerland': 'CH', 'Taiwan': 'TW', 'Thailand': 'TH', 'Türkiye': 'TR', 'Ukraine': 'UA', 'United Kingdom': 'GB', 'United States': 'US', 'Vietnam': 'VN'}


In [13]:
json_data = json.dumps(
    nameToCodeOsuFiltered, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path_filter = "out/osuCountriesFilter.json"
with open(file_path_filter, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)

JSON data has been saved to out/osu_countries.json
